In [1]:
import imageio
from tqdm import tqdm
import multiprocessing as mp

In [2]:
%run ../utils/unet.py
%run ../utils/helpers.py
%run ../utils/generate_flat_bg.py
%run ../utils/generate_bg.py
%run ../utils/generate_train_set.py

Using TensorFlow backend.


In [3]:
h_size = 256
w_size = 256
start_neurons = 16
mask_pool_size = 10
train_num = 50
bg_admix_iters = 5

cores_count = mp.cpu_count()
cpu_pool = mp.Pool(processes=cores_count)

In [4]:
train_x = np.zeros((train_num, h_size, w_size, 3), dtype='float32')
train_y = np.zeros((train_num, h_size, w_size, 1), dtype='float32')

model = get_compiled_model(h_size, w_size, start_neurons)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
sample_img = imageio.imread('../data/train/original_sample_blur_1000.jpg', pilmode="RGB")
mask_img = imageio.imread('../data/train/original_sample_mask_inv_1000.jpg', pilmode="RGB", as_gray=True)

original_smpl = np.array(sample_img, dtype=np.uint8)
original_mask = np.array(mask_img, dtype=np.uint8)

In [6]:
simplest_flat_bg = imageio.imread('../data/train/simplest_flat_bg.jpg', pilmode="RGB")
simple_bg = imageio.imread('../data/train/simplest_bg.jpg', pilmode="RGB")

In [7]:
mask_pool = [
    get_mask(h_size, w_size) 
    for _ in tqdm(range(mask_pool_size))
]

100%|██████████| 10/10 [00:00<00:00, 12.96it/s]


In [8]:
%%time
for k in range(train_num): # tqdm
    img, msk = next_pair(
        original_mask,
        original_smpl,
        simplest_flat_bg,
        simple_bg,
        mask_pool,
    )
    train_x[k] = img
    train_y[k] = msk

CPU times: user 16.3 s, sys: 314 ms, total: 16.6 s
Wall time: 14.9 s


In [9]:
%%time
coros = [
    cpu_pool.apply_async(
        next_pair, 
        args=(
            original_mask,
            original_smpl,
            simplest_flat_bg,
            simple_bg,
            mask_pool,
        )
    ) 
    for _ in range(train_num)
]
for img, msk in [p.get() for p in coros]:
    train_x[k] = img
    train_y[k] = msk

CPU times: user 382 ms, sys: 809 ms, total: 1.19 s
Wall time: 7.49 s


In [10]:
train_x.shape

(50, 256, 256, 3)